# Optimizers

Implementations for SDG, Momentum, etc. up to AdamW and Muon

## SGD (Stochastic Gradient Descent)

**Idea:** Take a step in the negative gradient direction.

```
θ = θ - lr * g
```

That's it. Simple, but struggles with noisy gradients and ill-conditioned landscapes.

In [ ]:
import torch

In [ ]:
class SDG(torch.optim.Optimizer):
    def __init__(self, params, lr=0.01, weight_decay=0.01):
        defaults = dict(lr=lr, weight_decay=weight_decay)
        super().__init__(params, defaults)
    
    @torch.no_grad()
    def step(self):
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                # Weight Decay
                grad = p.grad
                if group['weight_decay'] != 0.0:
                    # equivalent to: loss = ... + weight_decay/2 * W**2
                    grad = grad.add(p, alpha=group['weight_decay'])  # not in place!
                # Update
                p.data.add_(grad, alpha=-group['lr'])


In [ ]:
# Seed
torch.manual_seed(42)

# Forward + backward
x = torch.randn(16, 32)
target = torch.randn(16, 64)

# Create identical weights
W1 = torch.randn(64, 32, requires_grad=True)
W2 = W1.clone().detach().requires_grad_(True)

# Params
lr = 0.02

# Optimizers
opt_torch = torch.optim.SGD([W1], lr=lr, weight_decay=0.01)
opt_custom = SDG([W2], lr=lr, weight_decay=0.01)

for i in range(20):
    opt_torch.zero_grad()
    opt_custom.zero_grad()
    loss1 = ((x @ W1.T - target) ** 2).mean()
    loss2 = ((x @ W2.T - target) ** 2).mean()
    loss1.backward()
    loss2.backward()

    opt_torch.step()
    opt_custom.step()

    weight_max_diff = (W1 - W2).abs().max().item()
    assert weight_max_diff == 0.0
print(f"All good after {i+1} iterations!")

## SGD with Momentum

**Idea:** Accumulate gradients over time into a "velocity." Smooths out noise, builds up speed in consistent directions.

```
v = β * v + g
θ = θ - lr * v
```

Typical β = 0.9 (averages ~10 steps). Steady-state velocity is g/(1-β), so effective step is larger than vanilla SGD.

In [ ]:
import torch

In [ ]:
class SDGMomentum(torch.optim.Optimizer):
    def __init__(self, params, lr=0.01, momentum=0.9, weight_decay=0.01):
        defaults = dict(lr=lr, momentum=momentum, weight_decay=weight_decay)
        super().__init__(params, defaults)
    
    @torch.no_grad()
    def step(self):
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                # Lazy Init
                if p not in self.state:
                    self.state[p] = {
                        'momentum_buffer': torch.zeros_like(p),
                    }
                # Weight Decay
                grad = p.grad
                if group['weight_decay'] != 0.0:
                    # equivalent to: loss = ... + weight_decay/2 * W**2
                    grad = grad.add(p, alpha=group['weight_decay'])  # not in place!
                # Update Step
                v = self.state[p]['momentum_buffer']
                v.mul_(group['momentum']).add_(grad)
                p.data.add_(v, alpha=-group['lr'])

In [ ]:
# Seed
torch.manual_seed(42)

# Forward + backward
x = torch.randn(16, 32)
target = torch.randn(16, 64)

# Create identical weights
W1 = torch.randn(64, 32, requires_grad=True)
W2 = W1.clone().detach().requires_grad_(True)

# Params
lr = 0.02

# Optimizers
opt_torch = torch.optim.SGD([W1], lr=lr, momentum=0.9, weight_decay=0.01)
opt_custom = SDGMomentum([W2], lr=lr, momentum=0.9, weight_decay=0.01)

for i in range(20):
    opt_torch.zero_grad()
    opt_custom.zero_grad()
    loss1 = ((x @ W1.T - target) ** 2).mean()
    loss2 = ((x @ W2.T - target) ** 2).mean()
    loss1.backward()
    loss2.backward()

    opt_torch.step()
    opt_custom.step()

    state1 = opt_torch.state[W1]
    state2 = opt_custom.state[W2]
    assert list(state1.keys()) == ['momentum_buffer']
    assert torch.equal(state1['momentum_buffer'], state2['momentum_buffer'])

    weight_max_diff = (W1 - W2).abs().max().item()
    assert weight_max_diff == 0.0
    print(f"Diff {weight_max_diff}")

print(f"All good after {i+1} iterations!")

## RMSprop

**Idea:** Fix AdaGrad by using EMA instead of sum. Old gradients decay away.

```
s = β * s + (1 - β) * g²
θ = θ - lr * g / (√s + ε)
```

Typical β = 0.99. Learning rate stabilizes instead of decaying to zero.

In [ ]:
import torch

In [ ]:
class RMSProp(torch.optim.Optimizer):
    def __init__(self, params, lr=0.01, alpha=0.99, eps=1e-8, weight_decay=0.01):
        defaults = dict(lr=lr, alpha=alpha, eps=eps, weight_decay=weight_decay)
        super().__init__(params, defaults)
    
    @torch.no_grad()
    def step(self):
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                # Lazy Init
                if p not in self.state:
                    self.state[p] = {
                        'square_avg': torch.zeros_like(p),
                    }

                # Weight Decay
                grad = p.grad
                if group['weight_decay'] != 0.0:
                    # equivalent to: loss = ... + weight_decay/2 * W**2
                    grad = grad.add(p, alpha=group['weight_decay'])  # not in place!

                # Update Step
                s = self.state[p]['square_avg']
                s.mul_(group['alpha'])

                # s = s + (1-group['alpha']) * grad * grad
                s.addcmul_(grad, grad, value=1-group['alpha'])

                avg = s.sqrt().add_(group['eps'])
                
                # p.add_(grad / avg, alpha=-group['lr'])
                p.data.addcdiv_(grad, avg, value=-group['lr'])

In [ ]:
# Seed
torch.manual_seed(42)

# Forward + backward
x = torch.randn(16, 32)
target = torch.randn(16, 64)

# Create identical weights
W1 = torch.randn(64, 32, requires_grad=True)
W2 = W1.clone().detach().requires_grad_(True)

# Params
lr = 0.02

# Optimizers
opt_torch = torch.optim.RMSprop([W1], lr=lr, alpha=0.99, weight_decay=0.01)
opt_custom = RMSProp([W2], lr=lr, alpha=0.99, weight_decay=0.01)

for i in range(20):
    opt_torch.zero_grad()
    opt_custom.zero_grad()
    loss1 = ((x @ W1.T - target) ** 2).mean()
    loss2 = ((x @ W2.T - target) ** 2).mean()
    loss1.backward()
    loss2.backward()

    opt_torch.step()
    opt_custom.step()

    state1 = opt_torch.state[W1]
    state2 = opt_custom.state[W2]
    assert list(state1.keys()) == ['step', 'square_avg']
    assert torch.equal(state1['square_avg'], state2['square_avg'])

    weight_max_diff = (W1 - W2).abs().max().item()
    assert weight_max_diff == 0.0
    print(f"Diff {weight_max_diff}")

print(f"All good after {i+1} iterations!")

## Adam

**Idea:** Combine momentum (first moment) with RMSprop (second moment). Add bias correction for early steps.

```
v = β1 * v + (1 - β1) * g          # first moment (direction)
s = β2 * s + (1 - β2) * g²         # second moment (scaling)

v_corrected = v / (1 - β1^t)       # bias correction
s_corrected = s / (1 - β2^t)

θ = θ - lr * v_corrected / (√s_corrected + ε)
```

Typical: β1 = 0.9, β2 = 0.999, ε = 1e-8.

Bias correction compensates for zero initialization. After ~1000 steps, correction ≈ 1.

In [ ]:
import torch

In [ ]:
class Adam(torch.optim.Optimizer):
    def __init__(self, params, lr=0.01, betas=(0.9, 0.999), eps=1e-8, weight_decay=0.01):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        super().__init__(params, defaults)
    
    @torch.no_grad()
    def step(self):
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                # Lazy Init
                if p not in self.state:
                    self.state[p] = {
                        'step': 0,
                        'exp_avg': torch.zeros_like(p),
                        'exp_avg_sq': torch.zeros_like(p),
                    }
                self.state[p]['step'] += 1

                # Weight Decay
                grad = p.grad
                if group['weight_decay'] != 0.0:
                    # equivalent to: loss = ... + weight_decay/2 * W**2
                    grad = grad.add(p, alpha=group['weight_decay'])  # not in place!

                # Update v
                # v = B1 * v + (1-B1) * g
                v = self.state[p]['exp_avg']
                v.lerp_(grad, 1 - group['betas'][0])

                # Update s
                # s = B2 * s + (1-B2) * g**2
                s = self.state[p]['exp_avg_sq']
                s.mul_(group['betas'][1])
                s.addcmul_(grad, grad, value=1-group['betas'][1])

                # Correction
                # Somewhat convoluted way to do:
                # v_corrected = v / (1-B1**t)
                # s_corrected = s / (1-B2**t)
                # p = p - lr * v_corrected / (sqrt(s_corrected)+eps)
                t = self.state[p]['step']
                bias1 = 1-group['betas'][0]**t
                bias2 = 1-group['betas'][1]**t
                bias2_sqrt = bias2**0.5
                denom = (s.sqrt() / bias2_sqrt).add_(group['eps'])
                p.data.addcdiv_(v, denom, value=-group['lr'] / bias1)



In [ ]:
# Seed
torch.manual_seed(42)

# Forward + backward
x = torch.randn(16, 32)
target = torch.randn(16, 64)

# Create identical weights
W1 = torch.randn(64, 32, requires_grad=True)
W2 = W1.clone().detach().requires_grad_(True)

# Params
lr = 0.02

# Optimizers
opt_torch = torch.optim.Adam([W1], lr=lr, betas=(0.9, 0.999), weight_decay=0.01)
opt_custom = Adam([W2], lr=lr, betas=(0.9, 0.999), weight_decay=0.01)

for i in range(20):
    opt_torch.zero_grad()
    opt_custom.zero_grad()
    loss1 = ((x @ W1.T - target) ** 2).mean()
    loss2 = ((x @ W2.T - target) ** 2).mean()
    loss1.backward()
    loss2.backward()

    opt_torch.step()
    opt_custom.step()

    state1 = opt_torch.state[W1]
    state2 = opt_custom.state[W2]
    assert list(state1.keys()) == ['step', 'exp_avg', 'exp_avg_sq']
    assert state1['step'] == state2['step']
    assert torch.equal(state1['exp_avg'], state2['exp_avg'])
    assert torch.equal(state1['exp_avg_sq'], state2['exp_avg_sq'])

    weight_max_diff = (W1 - W2).abs().max().item()
    assert weight_max_diff == 0.0
    print(f"Diff {weight_max_diff}")

print(f"All good after {i+1} iterations!")

In [ ]:
weight_max_diff

In [ ]:
state2['exp_avg']